In [3]:
import pandas as pd
import numpy as np
from metrics_forecast_class import TimeSeriesEvaluator,MultiSeriesEvaluator
h=6
window=24
sources=['instagram-content','twitter-tweets','tiktok-content','facebook-content','youtube-content',
 'tiktok-content-oauth','instagram-content-oauth','tiktok-content-public']
df= pd.read_csv('~/rug/thesis/thesis/data/influencer_sample010525_transformed.csv').query('source in @sources').reset_index(drop=True)
df['published_date'] = pd.to_datetime(df['publishedDate'])
df


,post_uid,channel_uid,publishedDate,metrics,potentialReach,source,comments,video_views,engagements,video_plays,shares,published_date,engagements_rate,audience,tier
0,e55f59160f5139bb81ae67f11a78d056,004621c59168375c92abc5072b0e9448,2017-10-22T18:13:39.000Z,"[{""name"":""instagram_likes"",""value"":""187""},{""na...",16226.0,instagram-content,9,NaN,196,NaN,NaN,2017-10-22 18:13:39+00:00,0.012079,16226.0,"10,000–49,999"
1,86a2a199061c3830aa754a6170560c8c,004621c59168375c92abc5072b0e9448,2017-10-22T18:15:03.000Z,"[{""name"":""instagram_likes"",""value"":""128""},{""na...",16226.0,instagram-content,5,NaN,133,NaN,NaN,2017-10-22 18:15:03+00:00,0.008197,16226.0,"10,000–49,999"
2,653a7cc6905134d0841e46ced8e18331,004621c59168375c92abc5072b0e9448,2017-10-22T18:15:54.000Z,"[{""name"":""instagram_likes"",""value"":""154""},{""na...",16226.0,instagram-content,5,NaN,159,NaN,NaN,2017-10-22 18:15:54+00:00,0.009799,16226.0,"10,000–49,999"
3,4084266d023333c69d5d8826cfe3642c,004621c59168375c92abc5072b0e9448,2017-10-24T19:25:45.000Z,"[{""name"":""instagram_likes"",""value"":""87""},{""nam...",16226.0,instagram-content,0,NaN,87,NaN,NaN,2017-10-24 19:25:45+00:00,0.005362,16226.0,"10,000–49,999"
4,ac2f3022b5d83ebfb63459fd65a819b0,004621c59168375c92abc5072b0e9448,2017-10-24T19:25:53.000Z,"[{""name"":""instagram_likes"",""value"":""174""},{""na...",16226.0,instagram-content,1,NaN,175,NaN,NaN,2017-10-24 19:25:53+00:00,0.010785,16226.0,"10,000–49,999"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218720,44b27da896833fbba09d0924e22f1deb,ff0e1dd64d473456a9f38bfe01cf0a44,2025-02-20T03:07:07.000Z,"[{""name"":""instagram_likes"",""value"":""153""},{""na...",7740.0,instagram-content,0,NaN,153,4366.0,0.0,2025-02-20 03:07:07+00:00,0.019767,7740.0,"1,000–9,999"
218721,3513ddf5321c3bb8b32eb14827b9fb20,ff0e1dd64d473456a9f38bfe01cf0a44,2025-02-25T04:24:46.000Z,"[{""name"":""instagram_comments"",""value"":""9""},{""n...",7740.0,instagram-content,0,NaN,59,NaN,0.0,2025-02-25 04:24:46+00:00,0.007623,7740.0,"1,000–9,999"
218722,7b2586860aeb3fa4bbca24ec536d99ca,ff0e1dd64d473456a9f38bfe01cf0a44,2025-03-04T17:08:44.000Z,"[{""name"":""instagram_likes"",""value"":""17""},{""nam...",7758.0,instagram-content,0,NaN,17,925.0,0.0,2025-03-04 17:08:44+00:00,0.002191,7758.0,"1,000–9,999"
218723,a0cdf63fc9e938e8a17c6f87da4b6d51,ff0e1dd64d473456a9f38bfe01cf0a44,2025-03-09T15:38:11.000Z,"[{""name"":""instagram_comments"",""value"":""2""},{""n...",7758.0,instagram-content,0,NaN,13,575.0,0.0,2025-03-09 15:38:11+00:00,0.001676,7758.0,"1,000–9,999"


In [4]:
df['source'].value_counts()

source
instagram-content    119688
twitter-tweets        50484
tiktok-content        36721
facebook-content       9096
youtube-content        2736
Name: count, dtype: int64

In [5]:
df.groupby('channel_uid').count()['publishedDate'].sort_values(ascending=False).head(10)

channel_uid
26bf145cbe9c3a10a53c619f05aa750f    32490
1058025cd111366f9d4d4af89ce1d8d0    24051
954328679b923a4fa7af327a2cb5b84d    13640
da9acec7669f3db79050175948471cde     9714
e04c6702e00a321a8f98624ffe7fd23e     6232
24a13f11774b37b2bb94e9ab25327bf7     4884
e6981573da773e0eb0493a0cf142e27c     4274
428271c836453fc2a6fd57748371c14d     3608
1fc2d888a1d63daea81ab67327629a64     3478
570ef814e5983fc8a152009d2be91bef     3352
Name: publishedDate, dtype: int64

In [6]:
ids_test=df.query("""source in ['instagram-content','youtube-content','tiktok-content','facebook-content']""")\
    ['channel_uid'].drop_duplicates().sample(1, random_state=1).values
#df_temp=df.query('channel_uid in @ids_test')
df_temp=df.copy()
df_temp['source'].unique()

array(['instagram-content', 'youtube-content', 'tiktok-content',
       'twitter-tweets', 'facebook-content'], dtype=object)

In [7]:
multi_eval = MultiSeriesEvaluator(df_temp, date_col='published_date', channel_col='channel_uid')

In [8]:

multi_eval.run(h=6, initial_window=24, step_length=6,models=['MA(5)'],metric_cols=['engagements_rate'])

summary_df = multi_eval.get_summary()
summary_complete_df = multi_eval.get_summary_complete()

Evaluating models:   0%|          | 0/189 [00:00<?, ?it/s]2025-05-12 12:20:02,277 Evaluating channel: 004621c59168375c92abc5072b0e9448
2025-05-12 12:20:02,327 Evaluating metric: engagements_rate
2025-05-12 12:20:02,331 Evaluating MA(5) for 004621c59168375c92abc5072b0e9448/engagements_rate
/home/esteban/rug/thesis/thesis/venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/esteban/rug/thesis/thesis/venv/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/esteban/rug/thesis/thesis/venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/esteban/rug/th

In [9]:
import datetime as dt
summary_df.to_csv(f'../output/summary_df_{dt.datetime.now():%y%m%d%H%M}.csv', index=False)
summary_df

,model_name,test_MeanAbsoluteError,test_MeanAbsolutePercentageError,test_MeanSquaredError,channel_uid,metric_name,source
0,MA(5),0.002524,1.253794,0.002753,004621c59168375c92abc5072b0e9448,engagements_rate,instagram-content
1,MA(5),0.126277,15.493182,0.144038,03bbf13a212736adaa05fde51f363d5e,engagements_rate,youtube-content
2,MA(5),0.004736,0.412213,0.005660,05fe8678b3c432c1834a8249b9b5fc2a,engagements_rate,instagram-content
3,MA(5),0.000127,0.390292,0.000138,0811a952ce6636fdb283cca1fd6b601a,engagements_rate,tiktok-content
4,MA(5),0.011417,0.417595,0.014233,08c242dca64f305598abbd56e7b22c58,engagements_rate,instagram-content
...,...,...,...,...,...,...,...
177,MA(5),0.041982,2.703092,0.056874,f477a13910a236b7b4bb1381ea6630bc,engagements_rate,youtube-content
178,MA(5),0.019582,0.911408,0.023048,f7674e4b4d7d3e3c90c7490028b5345a,engagements_rate,instagram-content
179,MA(5),0.011535,0.611471,0.014503,f84b3aaef6583cb9bd11ec8a859e5810,engagements_rate,youtube-content
180,MA(5),0.007469,1.414025,0.008726,fb001ff538ac3461885949bf941bf5fa,engagements_rate,instagram-content


In [10]:

summary_complete_df.to_csv(f'../output/summary_complete_df_{dt.datetime.now():%y%m%d%H%M}.csv', index=False)
summary_complete_df

,test_MeanAbsoluteError,test_MeanSquaredError,test_MeanAbsolutePercentageError,fit_time,pred_time,len_train_window,cutoff,y_train,y_test,y_pred,model_name,channel,metric_name,source
0,0.003510,0.003675,1.371262,3.321783,0.099398,24,23,0 0.012079 1 0.008197 2 0.009799 3...,24 0.003390 25 0.002157 26 0.002465 2...,24 0.005880 25 0.006572 26 0.006523 2...,MA(5),004621c59168375c92abc5072b0e9448,engagements_rate,instagram-content
1,0.002975,0.003104,1.641372,1.407257,0.028115,30,29,0 0.012079 1 0.008197 2 0.009799 3...,30 0.002034 31 0.002342 32 0.002404 3...,30 0.004041 31 0.004504 32 0.005465 3...,MA(5),004621c59168375c92abc5072b0e9448,engagements_rate,instagram-content
2,0.001979,0.002212,0.954060,1.291748,0.021829,36,35,0 0.012079 1 0.008197 2 0.009799 3...,36 0.004745 37 0.004314 38 0.005793 3...,36 0.003771 37 0.003659 38 0.004315 3...,MA(5),004621c59168375c92abc5072b0e9448,engagements_rate,instagram-content
3,0.001632,0.002022,1.048482,1.761807,0.035227,42,41,0 0.012079 1 0.008197 2 0.009799 3...,42 0.001294 43 0.001787 44 0.001233 4...,42 0.004698 43 0.004536 44 0.003370 4...,MA(5),004621c59168375c92abc5072b0e9448,engagements_rate,instagram-content
4,0.126277,0.144038,15.493182,2.088227,0.020248,24,23,0 0.069672 1 0.050546 2 0.020492 3...,24 0.009353 25 0.016197 26 0.027465 2...,24 0.033594 25 0.126482 26 0.132152 2...,MA(5),03bbf13a212736adaa05fde51f363d5e,engagements_rate,youtube-content
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,0.011581,0.013488,1.603792,0.538799,0.025432,42,41,0 0.001332 1 0.001493 2 0.001391 3...,42 0.003044 43 0.027927 44 0.021341 4...,42 0.014789 43 0.003035 44 0.010939 4...,MA(5),fb001ff538ac3461885949bf941bf5fa,engagements_rate,instagram-content
716,0.009337,0.011526,0.340096,0.716456,0.017854,24,23,0 0.038680 1 0.038325 2 0.015791 3...,24 0.018808 25 0.016501 26 0.026260 2...,24 0.006097 25 0.019269 26 0.019442 2...,MA(5),ff0e1dd64d473456a9f38bfe01cf0a44,engagements_rate,instagram-content
717,0.016195,0.018168,1.170378,0.767299,0.016865,30,29,0 0.038680 1 0.038325 2 0.015791 3...,30 0.011888 31 0.014549 32 0.008517 3...,30 0.035489 31 0.037284 32 0.029305 3...,MA(5),ff0e1dd64d473456a9f38bfe01cf0a44,engagements_rate,instagram-content
718,0.008411,0.011054,0.320878,0.496879,0.016882,36,35,0 0.038680 1 0.038325 2 0.015791 3...,36 0.022179 37 0.054826 38 0.027857 3...,36 0.033578 37 0.033025 38 0.026364 3...,MA(5),ff0e1dd64d473456a9f38bfe01cf0a44,engagements_rate,instagram-content
